In [ ]:
%reset -fs
import pandas as pd
import  numpy
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from dask import dataframe as dd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
#Dask !!!
ddf = dd.read_csv(r"C:\Users\nated\Downloads\CSV-01-12\01-12\DrDoS_LDAP.csv", dtype={'SimillarHTTP': 'object'},blocksize='64MB')
df = ddf.compute()

In [ ]:
df.head()

In [ ]:
df.columns = df.columns.str.replace(' ', '')
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
(df['Label'].value_counts()) / len(df) * 100

In [ ]:
df['Unnamed:0'].nunique()

In [ ]:
df.drop(columns=['FlowID', 'SourceIP', 'DestinationIP', 'Timestamp', 'SimillarHTTP'], inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.fillna(0)

In [ ]:
df = pd.get_dummies(df, prefix = 'is_', columns='Label')
df

In [ ]:
df_working = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(frac=.005))

In [ ]:
df_working

In [ ]:
(df_working['Label'].value_counts()) / len(df_working) * 100

In [ ]:
df_working.shape

In [ ]:
n = df_working.nunique(axis=0)
  
print("No.of.unique values in each column :\n",
      n)

In [ ]:
# Alright these dataframes are taking up like 60% of my 32 GB of RAM so lets brick this nonsense and start over
df_working.to_csv('working_data.csv')

In [ ]:
%reset -fs
import pandas as pd
import  numpy
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from dask import dataframe as dd
from dask.diagnostics import ProgressBar
import sqlite3
import csv
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
df = pd.read_csv('working_data.csv', low_memory=False)

In [ ]:
df.head()

In [ ]:
# Much better memory usage
df.info()

In [ ]:
def drop_columns():
    n = df_working.nunique(axis=0)
